# welcome to the native nix jupyter kernel

this notebook shows what the `xeus-lix` kernel can do. it's a c++ jupyter kernel for the nix language, powered by the lix evaluator. you can run nix code, manage your environment, build things, and use the shell, all inside a notebook.

## 1. basic nix expressions

you can run multiple standard nix expressions in one cell

In [1]:
100 * 5

"Hello, " + "Jupyter!"

[ 1 2 3 4 ]

rec {
  x = "Nix";
  y = "Kernel";
  greeting = "Welcome to the ${x} ${y}!";
}.greeting

500
"Hello, Jupyter!"
[
  1
  2
  3
  4
]


"Welcome to the Nix Kernel!"

## 2. variables and functions

the kernel keeps state between cells. you can define variables and functions and use them later. the syntax is the same as the normal nix repl.

In [2]:
mySet = { a = 10; b = 20; }

In [3]:
mySet.a + mySet.b

30

In [4]:
add = x: y: x + y

In [5]:
add 5 15

20

## 3. interacting with the kernel (repl commands)

the kernel supports repl-style commands that start with a colon (`:`). these help you look at your environment and get docs.

In [6]:
:help

The following commands are available:

```
  <expr>                       Evaluate and print expression
  <x> = <expr>                 Bind expression to variable
  :a, :add <expr>              Add attributes from resulting set to scope
  :b <expr>                    Build a derivation
  :bl <expr>                   Build a derivation, creating GC roots
                               in the working directory
  :env                         Show variables in the current scope
  :doc <expr>                  Show documentation for the provided value
  :l, :load <path>             Load Nix expression and add it to scope
  :lf, :load-flake <ref>       Load Nix flake and add it to scope
  :p, :print <expr>            Evaluate and print expression recursively
                               Strings are printed directly, without escaping.
  :r, :reload                  Reload all files
  :t <expr>                    Describe result of evaluation
  :log <expr | .drv path>      Show logs for a derivation
  :te, :trace-enable [bool]    Enable, disable or toggle showing traces for
                               errors
  :?, :help                    Brings up this help menu
```


In [7]:
:t add

a function


In [8]:
:doc builtins.map

**Synopsis:** `builtins.map` *f* *list*

Apply the function *f* to each element in the list *list*. For
example,

```nix
map (x: "foo" + x) [ "bar" "bla" "abc" ]
```

evaluates to `[ "foobar" "foobla" "fooabc" ]`.


In [9]:
# Display all user-accessible variables in the current environment
:env

Current scope variables:
  add
  mySet

(Note: This is a simplified view of the top-level scope)


## 4. shell and filesystem interaction

you can run any shell command by starting it with `!`. the `:load` command lets you bring in definitions from other `.nix` files.

In [10]:
!echo "Hello from the shell running inside the Nix kernel!"
!date

Hello from the shell running inside the Nix kernel!
Sun Jul 13 15:13:28 CEST 2025


In [11]:
# Create a file named 'test.nix' to load
!echo '{ message = "hello from file"; value = 42; }' > test.nix

:load ./test.nix

Added 2 variables.


In [12]:
"The message is: '${message}' and the value is: ${toString value}"

"The message is: 'hello from file' and the value is: 42"

## 5. managing nixpkgs and flakes

loading all of `nixpkgs` or other flakes is easy.

In [13]:
# first, load nixpkgs into the scope
:l <nixpkgs>

# now, use a 'with' statement
with pkgs; lib.version

Added 25104 variables.


"25.11pre824823.c860cf0b3a08"

In [14]:
# load the official nixpkgs flake
:lf nixpkgs

lib.version

fetching github input 'github:NixOS/nixpkgs/nixpkgs-unstable'
Added 17 variables.


"25.11.20250708.2a21304"

## 6. building derivations

you can build derivations right away with the `:b` (build) and `:bl` (build local) commands. you can see the build logs with `:log`

In [15]:
# define a simple derivation
helloDerivation = pkgs.runCommand "hello" {} ''
  mkdir $out
  echo "Hello from a Nix derivation!" > $out/hello.txt
''

# build it using :b
:b helloDerivation

Building /nix/store/pmi91q85ksab7g1d7r3kd4hsjp5zjkll-hello.drv

This derivation produced the following outputs:
  out -> /nix/store/zxlhc27dzg2zixrd8hr4rivirr1gdxll-hello


In [16]:
# define another derivation
localDrv = pkgs.runCommand "my-local-build" {} ''
  mkdir -p $out/bin
  echo "#!/bin/sh" > $out/bin/greet
  echo "echo Hello from the :bl command!" >> $out/bin/greet
  chmod +x $out/bin/greet
'';

# build it and create a local symlink using :bl
:bl localDrv

Building /nix/store/hcflzfx9yb7019ldx7rc67jvdvjfmbrs-my-local-build.drv

This derivation produced the following outputs:
  ./result-out -> /nix/store/g10kvyyxzlxhmp98g6q735fqxv64bzp2-my-local-build


In [17]:
# check for the 'result-out' symlink in the current directory
!ls -l result-out

# execute the script from the build output via the symlink
!./result-out/bin/greet

lrwxr-xr-x 1 ellie staff 58 Jul 13 15:13 result-out -> /nix/store/g10kvyyxzlxhmp98g6q735fqxv64bzp2-my-local-build
Hello from the :bl command!


In [18]:
# get the build log for a package from nixpkgs
:log pkgs.neofetch

Log for /nix/store/fw0f9kkiwv5bayh09d9bdzy30s1khb3r-neofetch-unstable-2021-12-10.drv from https://cache.nixos.org:
Running phase: unpackPhase
unpacking source archive /nix/store/lanzxvdspljq3frjc67i16rfiig7p1aa-source
source root is source
Running phase: patchPhase
applying patch /nix/store/l2ypxlh4kc58qzi4zrvjry9516n4kmbm-avoid_overwriting_gio_extra_modules_env_var.patch
patching file neofetch
applying patch /nix/store/qglbm55x5jk9va9l74ky0203rk3lkkxi-update_old_nixos_logo.patch
patching file neofetch
applying patch /nix/store/7n3jbh84r1q7gb46cijlpx4x53nq4mp8-improve_detect_nixos_version.patch
patching file neofetch
patching script interpreter paths in neofetch
neofetch: interpreter directive changed from "#!/usr/bin/env bash" to "/nix/store/nqh4lv0w7n7cw7n8vvn2gfzv804y4hk6-bash-interactive-5.2p37/bin/bash"
Running phase: updateAutotoolsGnuConfigScriptsPhase
Running phase: configurePhase
no configure script, doing nothing
Running phase: buildPhase
build flags: SHELL=/nix/store/nravshv

## 7. advanced features

the kernel also has pretty-printing, debugging with `trace`, and rich display output.

In [19]:
builtins.trace "Hello World" 123

trace: Hello World


123

In [20]:
# define a nested attribute set with a derivation (thunk)
myNestedSet = {
  a = 1;
  b = {
    c = "hello";
    d = [ 1 2 { e = "deep"; } ];
  };
  f = pkgs.hello; # this is a thunk that should not be deeply evaluated
}

In [21]:
# use :p to recursively print the entire structure without forcing thunks
:p myNestedSet

{
  a = 1;
  b = {
    c = "hello";
    d = [
      1
      2
      { e = "deep"; }
    ];
  };
  f = «derivation /nix/store/x6p76aiji2i202jdalv98ha8qkrqv9kv-hello-2.12.2.drv»;
}


In [22]:
{
  # the _toMime attribute signals to the kernel how to display this value
  _toMime = {
    # for binary data like images, it must be a base64 encoded string
    "image/png" = "iVBORw0KGgoAAAANSUhEUgAAAAgAAAAIAQMAAAD+wSzIAAAABlBMVEX///+/v7+jQ3Y5AAAADklEQVQI12P4AIX8EAgALgAD/aNpbtEAAAAASUVORK5CYII=";
    
    # it's good practice to include a text/plain fallback
    "text/plain" = "A tiny 8x8 red and gray PNG image.";
  };
}

## 8. error handling

the kernel correctly catches and shows different errors from parsing and evaluation.

In [23]:
# an undefined variable error
"valid expression before an error"
let x = 1; in y

"valid expression before an error"


UndefinedVarError: [31;1merror:[0m undefined variable '[35;1my[0m'
       [34;1mat [35;1m«string»:1:15[0m:
            1| let x = 1; in y
             |               [31;1m^[0m
            2|

In [ ]:
# a syntax error
let x = ; in x

In [ ]:
# an evaluation error (missing attribute)
pkgs.lib.nonExistentAttribute

In [ ]:
# an incomplete multiline string parse error
'''
amogus